In [1]:
import pandas as pd
import numpy as np

#Resampling data 

#loading data
df_female = pd.read_csv('test_data2/df_female.csv')
df_age = pd.read_csv('test_data2/df_age.csv')
df_SFI = pd.read_csv('test_data2/df_SFI.csv')
df_uti = pd.read_csv('test_data2/df_uti.csv')
df_admissions = pd.read_csv('test_data2/df_admissions.csv')
df_FIM_total = pd.read_csv('test_data2/df_FIM_total.csv')
df_acute_days = pd.read_csv('test_data2/df_acute_days.csv')
df_brain_injury = pd.read_csv('test_data2/df_brain_injury_mod.csv')

#Removing extra column
df_female = df_female.loc[:, ~df_female.columns.str.contains('^Unnamed')]
df_age = df_age.loc[:, ~df_age.columns.str.contains('^Unnamed')]
df_SFI = df_SFI.loc[:, ~df_SFI.columns.str.contains('^Unnamed')]
df_uti = df_uti.loc[:, ~df_uti.columns.str.contains('^Unnamed')]
df_admissions = df_admissions.loc[:, ~df_admissions.columns.str.contains('^Unnamed')]
df_FIM_total = df_FIM_total.loc[:, ~df_FIM_total.columns.str.contains('^Unnamed')]
df_acute_days = df_acute_days.loc[:, ~df_acute_days.columns.str.contains('^Unnamed')]
df_brain_injury = df_brain_injury.loc[:, ~df_brain_injury.columns.str.contains('^Unnamed')]


#rename FIM_total to value (column-name needs to be value for timeseriesflattener)
df_FIM_total = df_FIM_total.rename(columns= {'FIM_total': 'value'})

In [ ]:
#Creating uti df with negative cases as well
merged = pd.merge(df_age, df_uti, on=['ID'], how='left')

merged = merged[['ID','value']]

#turning UVI-neg cases from nan to 0
where_are_NaNs = np.isnan(merged)
merged[where_are_NaNs] = 0

#checking
print(merged)

#Changing from floats to int
merged['value'] = merged['value'].astype('int')

merged = pd.merge(merged, df_uti, on=['ID', 'value'], how='left')

#print(merged)

print(len(merged['value'])) #1864 patients in total
print(sum(merged['value'])) #518 patients get UTI

In [ ]:
df_SFI

In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta
import warnings

random.seed(42)

warnings.filterwarnings("ignore", category=FutureWarning) # Doesnt really fix the issue that the function append will be deprecated later. however I have not been able to find solution.
warnings.filterwarnings("ignore", category=DeprecationWarning)
no_uti = 0
uti = 0
not_enough_rows=0
no_date_list = 0
columns = ['ID', 'date', 'note']

X_out = pd.DataFrame(columns=columns)

columns2 = ['ID', 'prediction_date']
prediction_times = pd.DataFrame(columns=columns2)

df_admissions['date'] = pd.to_datetime(df_admissions['date'], format = "%Y-%m-%d %H:%M:%S")
df_SFI['date'] = pd.to_datetime(df_SFI['date'], format = "%Y-%m-%d %H:%M:%S")
df_uti['date'] = pd.to_datetime(df_uti['date'], format = "%Y-%m-%d %H:%M:%S")

for ID in df_SFI['ID'].unique():
    SFI_df_sub = df_SFI[df_SFI['ID'] == ID]
    df_admissions_sub = df_admissions[df_admissions['ID'] == ID]

    if ID in df_uti['ID'].values:
        uti_date = df_uti.loc[df_uti['ID'] == ID, 'date'].iloc[0]
    else: 
        uti_date = 'NULL'
    
    #if they never get a UTI, select four random consecutive days of their admission
    if uti_date == 'NULL':
        no_uti += 1
        rows = len(df_admissions_sub)
        
        if rows < 5:
            not_enough_rows +=1 #patients who are admitted for four days or less
            continue
        else:
            n = random.randint(0, rows-5)
            
            df_pred_days = df_admissions_sub[n:n+5]
            prediction_date = df_pred_days.iloc[-1,0]
            prediction_time = pd.DataFrame({'ID': ID, 'prediction_date': [prediction_date]}) 
            #print(df_pred_days)
            date_list = []
        
            for date in SFI_df_sub['date']:
                if df_pred_days.iloc[0,0] <= date <  df_pred_days.iloc[-1,0]:
                    date_list.append(date)

            sample_df = SFI_df_sub[SFI_df_sub['date'].isin(date_list)]
            if sample_df.empty: 
                no_date_list += 1

            
    else:
        date_list = []
        uti += 1
        sample_df = pd.Series()
        prediction_date = uti_date-timedelta(days=1)
        prediction_time = pd.DataFrame({'ID': ID, 'prediction_date': [prediction_date]})     
        

        for date in SFI_df_sub['date']: 
            if uti_date-timedelta(days=5) <= date < uti_date:
                print(date, uti_date)
                date_list.append(date)


        sample_df = SFI_df_sub[SFI_df_sub['date'].isin(date_list)]
        if sample_df.empty: 
            no_date_list += 1

    #X_out = pd.merge(X_out, sample_df, on=['ID'])
    prediction_times = pd.concat([prediction_times, prediction_time], ignore_index=True, axis=0)
    X_out = pd.concat([X_out, sample_df], ignore_index=True, axis=0)

In [ ]:
print(len(X_out['ID'].unique())) #984
print(no_uti) #1346
print(uti) #518
print(not_enough_rows)#11 patients who are admitted for less than four days
print(no_date_list) #868 patients who dont have notes within the given period 


print(len(X_out['ID'].unique()) + no_date_list + not_enough_rows)
print(no_uti+uti)
len(prediction_times)


In [7]:
patients = X_out['ID'].unique()

#Filtering out patients who dont have notes within the specified dates
prediction_times = prediction_times.loc[prediction_times['ID'].isin(patients)]



In [ ]:
prediction_times

prediction_times = prediction_times.rename(columns ={"prediction_date" : "date"})
prediction_times

In [ ]:
X_out

#display(X_out.to_string())
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(X_out)

In [10]:
X_out.to_csv("test_data2/df_SFI_resampled.csv")
prediction_times.to_csv("test_data2/df_prediction_times_resampled.csv")